In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [5]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def get_mean_var(y_logit):
    y_logit=np.abs(y_logit-0.1)
    return np.mean(y_logit,axis=1)

def get_possibility(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=100, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
        y_logits=np.array(y_logits)
    return y_logits
import os
if not os.path.isdir('saved_models_{}'.format('lsgan')):
    os.mkdir('saved_models_{}'.format('lsgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('lsgan'), mode='w')
import torch.utils.data as Data
import cv2
x = []
y = np.zeros((31, 1), dtype=np.int)
y = list(y)
for i in range(31):
   y[i] = []
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv2D,GlobalAveragePooling2D,LeakyReLU,Conv2DTranspose
from keras.optimizers import Adam
from scipy import misc
import keras
from keras.datasets import cifar10
import os
import keras.backend.tensorflow_backend as KTF
import matplotlib.pyplot as plt
from scipy import misc
def set_gpu_config(device = "0",fraction=0.25):
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    config.gpu_options.visible_device_list = device
    KTF.set_session(tf.Session(config=config))


def predict_images(file_name, generator, noise_size, n = 10, size = 32):

    image = generator.predict(np.random.normal(size=(n*n, ) + noise_size))

    image = np.reshape(image, (n, n, size, size, 3))
    image = np.transpose(image, (0, 2, 1, 3, 4))
    image = np.reshape(image, (n*size, n*size, 3))

    image = 255 * (image + 1) / 2
    image = image.astype("uint8")
    misc.imsave(file_name, image)
def build_generator(input_shape):
    model = Sequential()

    model.add(Conv2DTranspose(512,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(3,(3,3),padding="same",activation="tanh"))
    model.summary()
    return model


def build_discriminator(input_shape):
    model = Sequential()

    model.add(Conv2D(64,(3,3),strides=(2,2),padding="same",input_shape=input_shape))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(128,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(256,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(512,(3,3),strides=(2,2),padding="same"))
    model.add(LeakyReLU(0.2))

    model.add(Conv2D(1,(3,3),padding="same"))
    model.add(GlobalAveragePooling2D())
    model.summary()

    return model

def build_functions(batch_size, noise_size, image_size, generator, discriminator):

    noise = K.random_normal((batch_size,) + noise_size,0.0,1.0,"float32")
    real_image = K.placeholder((batch_size,) + image_size)
    fake_image = generator(noise)

    LAMBA = 0.0002

    d_input = K.concatenate([real_image, fake_image], axis=0)
    pred_real, pred_fake = tf.split(discriminator(d_input), num_or_size_splits = 2, axis = 0)

    d_loss = K.mean(K.maximum(pred_real - pred_fake + LAMBA * K.sum(K.abs(real_image-fake_image),axis=[1,2,3]), 0.0))
    g_loss = K.mean(pred_fake)

    d_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(d_loss, discriminator.trainable_weights)
    d_train = K.function([real_image, K.learning_phase()], [d_loss], d_training_updates)

    g_training_updates = Adam(lr=0.0001, beta_1=0.0, beta_2=0.9).get_updates(g_loss, generator.trainable_weights)
    g_train = K.function([real_image, K.learning_phase()], [g_loss], g_training_updates)

    return d_train,g_train
def train():
    set_gpu_config("0", 0.5)

    epochs = 50
    image_size = (32, 32, 3)
    noise_size = (2, 2, 32)
    batch_size = 64
    sample_size = 10
    size = 32
    sample_interval=200
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    num_of_data = x_train.shape[0]
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    x_test = (x_test.astype(np.float32) - 127.5) / 127.5
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)

    generator = build_generator(noise_size)
    discriminator = build_discriminator(image_size)
    d_train, g_train = build_functions(batch_size, noise_size, image_size, generator, discriminator)

    nb_batches = int(x_train.shape[0] / batch_size)
    global_step = 0
    steps = []
    values = []
    for epoch in range(epochs):
        for index in range(nb_batches):
            global_step += 1
            real_images = x_train[index * batch_size:(index + 1) * batch_size]
            d_loss, = d_train([real_images, 1])
            g_loss, = g_train([real_images, 1])
            print("[{0}/{1}] [{2}_{3}] d_loss: {4:.4}, g_loss: {5:.4}".format(epoch, epochs, epoch, global_step, d_loss,
                                                                              g_loss))
            sampleSize = 5000
            # If at save interval => save generated image samples
            if global_step % sample_interval == 0:
                x.append(epoch)
                gen_imgs = generator.predict(np.random.normal(size=(sample_size,) + noise_size))
                gen_imgs= 255.0 * (gen_imgs + 1) / 2
                if global_step % sample_interval == 0:
                    y_logits = get_possibility(gen_imgs)
                    metrics = get_mean_var(y_logits)


                    f.writelines('\n')
                    f.writelines('global_step:' + str(global_step))
                    f.writelines('\n')
                    f.writelines(' %.8f ' % (i) for i in metrics)
                    f.writelines('\n')



if __name__ == '__main__':
    train()

Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): BatchNorm2d(512, eps=1e-05

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_13 (Conv2DT (None, 4, 4, 512)         147968    
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 4, 4, 512)         0         
_________________________________________________________________
conv2d_transpose_14 (Conv2DT (None, 8, 8, 256)         1179904   
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_15 (Conv2DT (None, 16, 16, 128)       295040    
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_16 (Conv2DT (None, 32, 32, 64)        73792     
__________

[0/50] [0_112] d_loss: 0.000144, g_loss: -0.4772
[0/50] [0_113] d_loss: 0.007981, g_loss: -0.08228
[0/50] [0_114] d_loss: 0.003029, g_loss: -0.04815
[0/50] [0_115] d_loss: 3.592e-06, g_loss: -0.1876
[0/50] [0_116] d_loss: 0.002494, g_loss: -0.2185
[0/50] [0_117] d_loss: 0.008126, g_loss: -0.2116
[0/50] [0_118] d_loss: 0.001439, g_loss: -0.4542
[0/50] [0_119] d_loss: 0.008093, g_loss: -0.4913
[0/50] [0_120] d_loss: 0.002495, g_loss: -0.3755
[0/50] [0_121] d_loss: 0.005083, g_loss: -0.1273
[0/50] [0_122] d_loss: 0.0002318, g_loss: -0.2874
[0/50] [0_123] d_loss: 0.0102, g_loss: -0.1886
[0/50] [0_124] d_loss: 0.003294, g_loss: -0.4524
[0/50] [0_125] d_loss: 0.002216, g_loss: -0.2667
[0/50] [0_126] d_loss: 2.511e-05, g_loss: -0.4082
[0/50] [0_127] d_loss: 0.0002332, g_loss: -0.4892
[0/50] [0_128] d_loss: 0.005902, g_loss: 0.167
[0/50] [0_129] d_loss: 0.0003521, g_loss: 0.2025
[0/50] [0_130] d_loss: 0.001309, g_loss: 0.2412
[0/50] [0_131] d_loss: 0.001402, g_loss: 0.001669
[0/50] [0_132] d_l

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[0/50] [0_201] d_loss: 0.005988, g_loss: -0.02826
[0/50] [0_202] d_loss: 0.002036, g_loss: -0.1526
[0/50] [0_203] d_loss: 5.534e-05, g_loss: -0.399
[0/50] [0_204] d_loss: 2.181e-05, g_loss: -0.7053
[0/50] [0_205] d_loss: 0.007587, g_loss: -0.7097
[0/50] [0_206] d_loss: 0.0, g_loss: -0.984
[0/50] [0_207] d_loss: 0.0, g_loss: -1.325
[0/50] [0_208] d_loss: 0.01167, g_loss: -0.4588
[0/50] [0_209] d_loss: 0.001429, g_loss: -0.2106
[0/50] [0_210] d_loss: 0.00601, g_loss: 0.3507
[0/50] [0_211] d_loss: 0.0001721, g_loss: 0.2822
[0/50] [0_212] d_loss: 0.0, g_loss: 0.0746
[0/50] [0_213] d_loss: 0.0, g_loss: -0.1207
[0/50] [0_214] d_loss: 0.003952, g_loss: -0.864
[0/50] [0_215] d_loss: 0.005929, g_loss: -0.746
[0/50] [0_216] d_loss: 0.0004164, g_loss: -0.6387
[0/50] [0_217] d_loss: 0.009578, g_loss: 0.69
[0/50] [0_218] d_loss: 0.004992, g_loss: -0.2581
[0/50] [0_219] d_loss: 6.728e-06, g_loss: -0.4688
[0/50] [0_220] d_loss: 0.01572, g_loss: 0.7142
[0/50] [0_221] d_loss: 0.0004425, g_loss: -0.1718

...

[49/50] [49_38338] d_loss: 0.09853, g_loss: 1.38
[49/50] [49_38339] d_loss: 0.135, g_loss: 1.488
[49/50] [49_38340] d_loss: 0.07949, g_loss: 1.335
[49/50] [49_38341] d_loss: 0.07012, g_loss: 1.251
[49/50] [49_38342] d_loss: 0.1146, g_loss: 1.171
[49/50] [49_38343] d_loss: 0.119, g_loss: 1.354
[49/50] [49_38344] d_loss: 0.05327, g_loss: 1.307
[49/50] [49_38345] d_loss: 0.1405, g_loss: 1.265
[49/50] [49_38346] d_loss: 0.06809, g_loss: 1.435
[49/50] [49_38347] d_loss: 0.115, g_loss: 1.293
[49/50] [49_38348] d_loss: 0.09413, g_loss: 1.459
[49/50] [49_38349] d_loss: 0.07462, g_loss: 1.49
[49/50] [49_38350] d_loss: 0.1337, g_loss: 1.421
[49/50] [49_38351] d_loss: 0.09768, g_loss: 1.346
[49/50] [49_38352] d_loss: 0.078, g_loss: 1.339
[49/50] [49_38353] d_loss: 0.08879, g_loss: 1.484
[49/50] [49_38354] d_loss: 0.07137, g_loss: 1.441
[49/50] [49_38355] d_loss: 0.1031, g_loss: 1.471
[49/50] [49_38356] d_loss: 0.06769, g_loss: 1.437
[49/50] [49_38357] d_loss: 0.1073, g_loss: 1.358
[49/50] [49_383

[49/50] [49_38507] d_loss: 0.1137, g_loss: 1.381
[49/50] [49_38508] d_loss: 0.1027, g_loss: 1.248
[49/50] [49_38509] d_loss: 0.1365, g_loss: 1.456
[49/50] [49_38510] d_loss: 0.107, g_loss: 1.103
[49/50] [49_38511] d_loss: 0.09451, g_loss: 1.24
[49/50] [49_38512] d_loss: 0.1038, g_loss: 1.255
[49/50] [49_38513] d_loss: 0.0913, g_loss: 1.182
[49/50] [49_38514] d_loss: 0.05945, g_loss: 1.227
[49/50] [49_38515] d_loss: 0.09518, g_loss: 1.198
[49/50] [49_38516] d_loss: 0.1096, g_loss: 1.302
[49/50] [49_38517] d_loss: 0.1139, g_loss: 1.23
[49/50] [49_38518] d_loss: 0.1269, g_loss: 1.387
[49/50] [49_38519] d_loss: 0.1237, g_loss: 1.418
[49/50] [49_38520] d_loss: 0.0959, g_loss: 1.362
[49/50] [49_38521] d_loss: 0.1204, g_loss: 1.386
[49/50] [49_38522] d_loss: 0.06669, g_loss: 1.405
[49/50] [49_38523] d_loss: 0.05817, g_loss: 1.224
[49/50] [49_38524] d_loss: 0.1241, g_loss: 1.092
[49/50] [49_38525] d_loss: 0.0777, g_loss: 1.135
[49/50] [49_38526] d_loss: 0.09072, g_loss: 1.458
[49/50] [49_38527

[49/50] [49_38675] d_loss: 0.06846, g_loss: 1.671
[49/50] [49_38676] d_loss: 0.07383, g_loss: 1.505
[49/50] [49_38677] d_loss: 0.09052, g_loss: 1.354
[49/50] [49_38678] d_loss: 0.1295, g_loss: 1.389
[49/50] [49_38679] d_loss: 0.1109, g_loss: 1.446
[49/50] [49_38680] d_loss: 0.149, g_loss: 1.185
[49/50] [49_38681] d_loss: 0.1173, g_loss: 1.343
[49/50] [49_38682] d_loss: 0.1065, g_loss: 1.556
[49/50] [49_38683] d_loss: 0.06349, g_loss: 1.3
[49/50] [49_38684] d_loss: 0.09373, g_loss: 1.425
[49/50] [49_38685] d_loss: 0.0639, g_loss: 1.312
[49/50] [49_38686] d_loss: 0.09339, g_loss: 1.538
[49/50] [49_38687] d_loss: 0.08541, g_loss: 1.29
[49/50] [49_38688] d_loss: 0.1099, g_loss: 1.38
[49/50] [49_38689] d_loss: 0.1005, g_loss: 1.458
[49/50] [49_38690] d_loss: 0.09072, g_loss: 1.432
[49/50] [49_38691] d_loss: 0.08079, g_loss: 1.637
[49/50] [49_38692] d_loss: 0.07925, g_loss: 1.558
[49/50] [49_38693] d_loss: 0.09764, g_loss: 1.495
[49/50] [49_38694] d_loss: 0.06724, g_loss: 1.479
[49/50] [49_3

[49/50] [49_38842] d_loss: 0.08876, g_loss: 1.244
[49/50] [49_38843] d_loss: 0.1025, g_loss: 1.133
[49/50] [49_38844] d_loss: 0.1014, g_loss: 1.246
[49/50] [49_38845] d_loss: 0.05911, g_loss: 1.177
[49/50] [49_38846] d_loss: 0.09347, g_loss: 1.358
[49/50] [49_38847] d_loss: 0.09753, g_loss: 1.159
[49/50] [49_38848] d_loss: 0.08674, g_loss: 1.213
[49/50] [49_38849] d_loss: 0.1185, g_loss: 1.211
[49/50] [49_38850] d_loss: 0.1406, g_loss: 1.332
[49/50] [49_38851] d_loss: 0.151, g_loss: 1.392
[49/50] [49_38852] d_loss: 0.07341, g_loss: 1.385
[49/50] [49_38853] d_loss: 0.0615, g_loss: 1.435
[49/50] [49_38854] d_loss: 0.08645, g_loss: 1.298
[49/50] [49_38855] d_loss: 0.1067, g_loss: 1.319
[49/50] [49_38856] d_loss: 0.07836, g_loss: 1.28
[49/50] [49_38857] d_loss: 0.0957, g_loss: 1.45
[49/50] [49_38858] d_loss: 0.05919, g_loss: 1.531
[49/50] [49_38859] d_loss: 0.1056, g_loss: 1.56
[49/50] [49_38860] d_loss: 0.07711, g_loss: 1.212
[49/50] [49_38861] d_loss: 0.07676, g_loss: 1.284
[49/50] [49_3

[49/50] [49_39010] d_loss: 0.1087, g_loss: 1.351
[49/50] [49_39011] d_loss: 0.1204, g_loss: 1.684
[49/50] [49_39012] d_loss: 0.1067, g_loss: 1.522
[49/50] [49_39013] d_loss: 0.1038, g_loss: 1.37
[49/50] [49_39014] d_loss: 0.1037, g_loss: 1.34
[49/50] [49_39015] d_loss: 0.07104, g_loss: 1.433
[49/50] [49_39016] d_loss: 0.07108, g_loss: 1.313
[49/50] [49_39017] d_loss: 0.06053, g_loss: 1.255
[49/50] [49_39018] d_loss: 0.1468, g_loss: 1.354
[49/50] [49_39019] d_loss: 0.1591, g_loss: 1.672
[49/50] [49_39020] d_loss: 0.1676, g_loss: 1.226
[49/50] [49_39021] d_loss: 0.08153, g_loss: 1.356
[49/50] [49_39022] d_loss: 0.1287, g_loss: 1.719
[49/50] [49_39023] d_loss: 0.09968, g_loss: 1.408
[49/50] [49_39024] d_loss: 0.08821, g_loss: 1.525
[49/50] [49_39025] d_loss: 0.04063, g_loss: 1.217
[49/50] [49_39026] d_loss: 0.09794, g_loss: 1.444
[49/50] [49_39027] d_loss: 0.08361, g_loss: 1.449
[49/50] [49_39028] d_loss: 0.06885, g_loss: 1.209
[49/50] [49_39029] d_loss: 0.1452, g_loss: 1.385
[49/50] [49_